# Portfolio Gambling Theory: Complete Example

This notebook demonstrates all major concepts from portfolio optimization and gambling theory.

## Table of Contents
1. Setup and Data Generation
2. Single-Period Optimization (Mean-Variance, Expected Utility, VaR)
3. Multi-Period Strategies (Kelly Criterion)
4. Gambling Strategies (Bold vs. Timid Play)
5. Comprehensive Comparison

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
np.random.seed(42)

print("Portfolio Gambling Theory Package")
print("=" * 50)
print("\nThis notebook demonstrates portfolio optimization")
print("criteria and gambling strategies.")

## 1. Generate Synthetic Market Data

We'll create 5 assets with different risk-return profiles.

In [ ]:
# Market parameters
n_assets = 5
n_periods = 252  # One year of daily data

# Define asset characteristics (annualized)
mean_returns_annual = np.array([0.08, 0.10, 0.12, 0.15, 0.18])
volatilities_annual = np.array([0.15, 0.18, 0.22, 0.28, 0.35])

# Convert to daily
mean_returns_daily = mean_returns_annual / 252
vol_daily = volatilities_annual / np.sqrt(252)

# Create correlation matrix
correlation = 0.3
corr_matrix = np.ones((n_assets, n_assets)) * correlation
np.fill_diagonal(corr_matrix, 1.0)

# Covariance matrix
vol_matrix = np.diag(vol_daily)
cov_matrix = vol_matrix @ corr_matrix @ vol_matrix

# Generate returns
from scipy.stats import multivariate_normal
returns = multivariate_normal.rvs(
    mean=mean_returns_daily,
    cov=cov_matrix,
    size=n_periods
)

# Create DataFrame
asset_names = [f'Asset_{i+1}' for i in range(n_assets)]
returns_df = pd.DataFrame(returns, columns=asset_names)

print("Generated Market Data:")
print("\nDaily Statistics:")
print(returns_df.describe())

print("\nAnnualized Statistics:")
ann_returns = returns_df.mean() * 252
ann_vols = returns_df.std() * np.sqrt(252)
print(pd.DataFrame({'Return': ann_returns, 'Volatility': ann_vols}))

## 2. Mean-Variance Optimization

Implement Markowitz portfolio optimization.

In [ ]:
def optimize_mean_variance(returns, risk_aversion=2.0):
    """
    Simple mean-variance optimization.
    Maximize: μ'w - (λ/2)w'Σw
    Subject to: w'1 = 1, w >= 0
    """
    import cvxpy as cp
    
    mu = np.mean(returns, axis=0)
    Sigma = np.cov(returns.T)
    n = len(mu)
    
    w = cp.Variable(n)
    portfolio_return = mu @ w
    portfolio_variance = cp.quad_form(w, Sigma)
    
    objective = cp.Maximize(
        portfolio_return - (risk_aversion / 2) * portfolio_variance
    )
    constraints = [cp.sum(w) == 1, w >= 0]
    
    problem = cp.Problem(objective, constraints)
    problem.solve()
    
    return w.value

# Optimize for different risk aversions
risk_aversions = [1.0, 2.0, 5.0]
mv_portfolios = {}

for gamma in risk_aversions:
    weights = optimize_mean_variance(returns, gamma)
    mv_portfolios[f'γ={gamma}'] = weights
    
    # Calculate metrics
    port_return = np.mean(returns @ weights) * 252
    port_vol = np.std(returns @ weights) * np.sqrt(252)
    sharpe = port_return / port_vol
    
    print(f"\nMean-Variance (γ={gamma}):")
    print(f"Weights: {weights}")
    print(f"Expected Return: {port_return:.2%}")
    print(f"Volatility: {port_vol:.2%}")
    print(f"Sharpe Ratio: {sharpe:.3f}")

## 3. Kelly Criterion

Maximize long-run growth rate: E[log(1 + r'w)]

In [ ]:
def optimize_kelly(returns, fraction=1.0):
    """
    Kelly criterion optimization.
    For small returns: w* ≈ Σ^(-1)μ
    """
    mu = np.mean(returns, axis=0)
    Sigma = np.cov(returns.T)
    
    try:
        Sigma_inv = np.linalg.inv(Sigma)
        weights = Sigma_inv @ mu
        
        # Normalize to sum to fraction
        weights = weights * fraction / np.sum(weights)
        weights = np.maximum(weights, 0)  # No short sales
        weights = weights / np.sum(weights)  # Renormalize
        
        return weights
    except:
        return np.ones(len(mu)) / len(mu)

# Full Kelly
kelly_weights = optimize_kelly(returns, fraction=1.0)
kelly_return = np.mean(returns @ kelly_weights) * 252
kelly_vol = np.std(returns @ kelly_weights) * np.sqrt(252)

print("\nKelly Criterion (Full):")
print(f"Weights: {kelly_weights}")
print(f"Expected Return: {kelly_return:.2%}")
print(f"Volatility: {kelly_vol:.2%}")

# Half Kelly (more conservative)
half_kelly_weights = optimize_kelly(returns, fraction=0.5)
half_kelly_return = np.mean(returns @ half_kelly_weights) * 252
half_kelly_vol = np.std(returns @ half_kelly_weights) * np.sqrt(252)

print("\nKelly Criterion (Half):")
print(f"Weights: {half_kelly_weights}")
print(f"Expected Return: {half_kelly_return:.2%}")
print(f"Volatility: {half_kelly_vol:.2%}")

## 4. Bold vs. Timid Play (Dubins-Savage)

For subfair games, bold play maximizes P(reach target).

In [ ]:
def simulate_bold_play(initial_wealth, target_wealth, win_prob, n_sims=10000):
    """
    Simulate bold play strategy.
    Bet everything you can each round.
    """
    successes = 0
    
    for _ in range(n_sims):
        wealth = initial_wealth
        
        while 0 < wealth < target_wealth:
            # Bold play: bet min(wealth, target - wealth)
            bet = min(wealth, target_wealth - wealth)
            
            if np.random.random() < win_prob:
                wealth += bet
            else:
                wealth -= bet
        
        if wealth >= target_wealth:
            successes += 1
    
    return successes / n_sims

def simulate_timid_play(initial_wealth, target_wealth, win_prob, 
                       min_bet=0.01, max_rounds=10000, n_sims=1000):
    """
    Simulate timid play strategy.
    Bet minimum amount each round.
    """
    successes = 0
    
    for _ in range(n_sims):
        wealth = initial_wealth
        rounds = 0
        
        while 0 < wealth < target_wealth and rounds < max_rounds:
            bet = min(min_bet, wealth)
            
            if np.random.random() < win_prob:
                wealth += bet
            else:
                wealth -= bet
            
            rounds += 1
        
        if wealth >= target_wealth:
            successes += 1
    
    return successes / n_sims

# Theoretical probability for bold play
def theoretical_bold_success(w, W, p):
    """P(reach W from w) for bold play in subfair game"""
    if p >= 0.5:
        return w / W
    q = 1 - p
    alpha = np.log(q) / np.log(q / p)
    return (w / W) ** alpha

# Run simulations
initial = 1.0
target = 2.0
win_prob = 0.49  # Subfair

bold_success = simulate_bold_play(initial, target, win_prob)
timid_success = simulate_timid_play(initial, target, win_prob)
theoretical = theoretical_bold_success(initial, target, win_prob)

print("\nGambling Strategies (Subfair Game, p=0.49):")
print("=" * 50)
print(f"Bold Play:")
print(f"  Empirical success rate: {bold_success:.3f}")
print(f"  Theoretical: {theoretical:.3f}")
print(f"\nTimid Play:")
print(f"  Empirical success rate: {timid_success:.3f}")
print(f"\nBold play achieves {bold_success/timid_success:.1f}x higher success rate!")

## 5. Comprehensive Comparison

Compare all strategies side-by-side.

In [ ]:
# Collect all portfolios
all_portfolios = {
    'Equal Weight': np.ones(n_assets) / n_assets,
    'MV (γ=2)': mv_portfolios['γ=2.0'],
    'Kelly': kelly_weights,
    'Half-Kelly': half_kelly_weights,
}

# Calculate metrics for each
results = []
for name, weights in all_portfolios.items():
    port_returns = returns @ weights
    
    results.append({
        'Strategy': name,
        'Return': np.mean(port_returns) * 252,
        'Volatility': np.std(port_returns) * np.sqrt(252),
        'Sharpe': (np.mean(port_returns) * 252) / (np.std(port_returns) * np.sqrt(252)),
        'Max_DD': np.min(np.minimum.accumulate(np.cumprod(1 + port_returns)) - 
                        np.cumprod(1 + port_returns)) / np.maximum.accumulate(np.cumprod(1 + port_returns)),
    })

results_df = pd.DataFrame(results)
print("\nStrategy Comparison:")
print("=" * 80)
print(results_df.to_string(index=False))

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Portfolio weights
ax = axes[0, 0]
x = np.arange(n_assets)
width = 0.2
for i, (name, weights) in enumerate(all_portfolios.items()):
    ax.bar(x + i * width, weights, width, label=name)
ax.set_xlabel('Asset')
ax.set_ylabel('Weight')
ax.set_title('Portfolio Weights')
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels(asset_names)
ax.legend()

# Returns
ax = axes[0, 1]
ax.bar(results_df['Strategy'], results_df['Return'])
ax.set_ylabel('Annual Return')
ax.set_title('Expected Returns')
ax.tick_params(axis='x', rotation=45)

# Volatility
ax = axes[1, 0]
ax.bar(results_df['Strategy'], results_df['Volatility'])
ax.set_ylabel('Annual Volatility')
ax.set_title('Risk (Volatility)')
ax.tick_params(axis='x', rotation=45)

# Sharpe Ratio
ax = axes[1, 1]
ax.bar(results_df['Strategy'], results_df['Sharpe'])
ax.set_ylabel('Sharpe Ratio')
ax.set_title('Risk-Adjusted Returns')
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Key Takeaways

1. **Mean-Variance**: Practical and interpretable, good for single-period optimization
2. **Kelly Criterion**: Maximizes long-run growth but can be very aggressive
3. **Half-Kelly**: Often best risk-adjusted returns (Sharpe ratio)
4. **Bold Play**: Counterintuitively optimal for subfair games with specific targets
5. **Choice depends on objectives**: No universally "best" strategy

## Next Steps

Explore the other notebooks for:
- Detailed single-period comparisons
- Kelly criterion deep dive
- Transaction costs and practical considerations
- Robust optimization techniques
- Real data applications